In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

import dicom

from swat import *

Load the available patient images.

In [2]:
IMAGE_PATH = '../sample_images'
patients = os.listdir(IMAGE_PATH)
patients.sort()

In [3]:
patients

['00cba091fa4ad62cc3200a657aeb957e',
 '0a099f2549429d29b32f349e95fb2244',
 '0a0c32c9e08cc2ea76a71649de56be6d',
 '0a38e7597ca26f9374f8ea2770ba870d',
 '0acbebb8d463b4b9ca88cf38431aac69',
 '0b20184e0cd497028bdd155d9fb42dc9',
 '0bd0e3056cbf23a1cb7f0f0b18446068',
 '0c0de3749d4fe175b7a5098b060982a1',
 '0c37613214faddf8701ca41e6d43f56e',
 '0c59313f52304e25d5a7dcf9877633b1',
 '0c60f4b87afcb3e2dfa65abbbf3ef2f9',
 '0c98fcb55e3f36d0c2b6507f62f4c5f1',
 '0c9d8314f9c69840e25febabb1229fa4',
 '0ca943d821204ceb089510f836a367fd',
 '0d06d764d3c07572074d468b4cff954f',
 '0d19f1c627df49eb223771c28548350e',
 '0d2fcf787026fece4e57be167d079383',
 '0d941a3ad6c889ac451caf89c46cb92a',
 '0ddeb08e9c97227853422bd71a2a695e',
 '0de72529c30fe642bc60dcb75c87f6bd']

Connect to the CAS server. My _authinfo file is located in my home directory: ~/_authinfo and contains as its first line "default user europe\sbxjen password ...".

In [ ]:
conn = CAS('rdcgrd001.unx.sas.com', '63901', authinfo='../../_authinfo')
conn.setsessopt(caslib='CASUSER(sbxjen)')

In [15]:
conn.loadactionset(actionset='image')
# conn.loadactionset(actionset='BioMedImage') seems not available
conn.loadactionset(actionset='deepLearn')

NOTE: Added action set 'image'.


ERROR: Action set 'BioMedImage' was not found.
ERROR: Action set 'BioMedImage' was not loaded due to errors.
ERROR: The action stopped due to errors.


NOTE: Added action set 'deepLearn'.


<div class="cas-results-key"><b>&#167; actionset</b></div>
<div class="cas-results-body">
<div>deepLearn</div>
</div>
<div class="cas-output-area"></div>
<p class="cas-results-performance"><small><span class="cas-elapsed">elapsed 0.0644s</span> &#183; <span class="cas-user">user 0.104s</span> &#183; <span class="cas-sys">sys 0.666s</span> &#183; <span class="cas-memory">mem 4.16MB</span></small></p>

In [8]:
def loadscan(path):
    """
    Load a scan, which consists of multiple slices.
    :param      path: path to a patient scan
    :return:    slices: a list of dicom.dataset.FileDataset
    """
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    try:
        st = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        st = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
    for s in slices:
        s.SliceThickness = st
    return slices

In [9]:
def getpixels(path):
    """
    Get scan pixels.
    :param path: 
    :return: 
    """
    slices = loadscan(path)
    image = np.stack([s.pixel_array for s in slices])
    return image

In [ ]:
Every image is a layered scan: a 3D (length x breadth x height) numpy array.

In [10]:
firstPatientPixels = getpixels(IMAGE_PATH + '/' + patients[0])
df = pd.DataFrame(firstPatientPixels[80])

In [11]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,...,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000
1,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,...,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000
2,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,...,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000
3,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,...,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000
4,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,...,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000,-2000


In [ ]:
Just a test to check how standard CAS images should look like.

In [12]:
conn.image.loadimages(casout={'caslib':'CASUSER(sbxjen)', 'name':'imagesTable', 'replace':True},
                      path='/u/etfcan/images/testImages')

NOTE: Loaded 4 images from /u/etfcan/images/testImages into Cloud Analytic Services table imagesTable.


<div class="cas-output-area"></div>
<p class="cas-results-performance"><small><span class="cas-elapsed">elapsed 0.959s</span> &#183; <span class="cas-user">user 1.54s</span> &#183; <span class="cas-sys">sys 4.65s</span> &#183; <span class="cas-memory">mem 488MB</span></small></p>

In [16]:
conn.image.flattenimagetable(casout={'caslib':'CASUSER(sbxjen)', 'name':'processedImagesTable', 'replace':True}, 
                                   imagetable={'caslib':'CASUSER(sbxjen)','name':'imagesTable'})
out = conn.table.fetch(table={'caslib':'CASUSER(sbxjen)', 'name':'processedImagesTable'}, to=2)

NOTE: Table IMAGESTABLE contains compressed images.
NOTE: 4 out of 4 images were processed successfully and saved to the Cloud Analytic Services table processedImagesTable.


In [17]:
out

,_path_,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c3063,c3064,c3065,c3066,c3067,c3068,c3069,c3070,c3071,c3072
0,/u/etfcan/images/testImages/sas3.jpg,253.0,197.0,142.0,254.0,194.0,140.0,254.0,197.0,136.0,...,132.0,47.0,150.0,124.0,56.0,149.0,119.0,62.0,151.0,135.0
1,/u/etfcan/images/testImages/sas1.jpg,208.0,119.0,58.0,220.0,117.0,54.0,225.0,119.0,52.0,...,37.0,90.0,57.0,38.0,94.0,62.0,43.0,91.0,59.0,40.0


In [ ]:
Just like in the execDemo2016Jan18_rev1_bak.html, we should be able to load dicom images now. 

In [22]:
conn.image.loadimages(cas Out={'name':'sample_images', 'replace':True}, decode=True, path='/u/sbxjen/sample_images/00cba091fa4ad62cc3200a657aeb957e')

NOTE: Loaded 0 images from /u/sbxjen/sample_images/00cba091fa4ad62cc3200a657aeb957e into Cloud Analytic Services table sample_images.


<div class="cas-output-area"></div>
<p class="cas-results-performance"><small><span class="cas-elapsed">elapsed 0.125s</span> &#183; <span class="cas-user">user 0.202s</span> &#183; <span class="cas-sys">sys 1.33s</span> &#183; <span class="cas-memory">mem 292MB</span></small></p>